In [1]:
// --- Day 13: Point of Incidence ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/13

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 13);
WriteLines(inputLines);

Loading puzzle file: Day13.txt
Total lines: 1349
Max line length: 17

######..#
.##....#.
.....##.#
.....##.#
.##..#.#.


In [4]:
IEnumerable<(T left, T right)> Mirror<T>(IList<T> input, int mirrorLength) {
    var left = input.Take(mirrorLength).Reverse();
    var right = input.Skip(mirrorLength).Take(mirrorLength);

    var pairs = left.Zip(right);

    return pairs;
}
Console.WriteLine(string.Join(",", Mirror([1,2,3,4,5,6,7], 5)));

(5, 6),(4, 7)


In [5]:
bool IsMirror<T>(IList<T> input, int mirrorLength, Func<T, T, bool> equality) => Mirror(input, mirrorLength).All(lr => equality(lr.left, lr.right));

In [6]:
using System.Collections.Immutable;

ImmutableHashSet<int> GetMirrors<T>(IList<T> input, Func<T, T, bool> equality) => Enumerable.Range(1, input.Count - 1)
    .Where(i => IsMirror(input, i, equality))
    .ToImmutableHashSet();

var charsEqual = (char left, char right) => left == right;

var testInputLine = "#.##..##.";
Console.WriteLine(String.Join(",", GetMirrors(testInputLine.ToList(), charsEqual)));

5,7


In [7]:
int GetColOfReflection(IEnumerable<string> inputs) => inputs
    .Select(input => GetMirrors(input.ToList(), charsEqual))
    .Aggregate((a, b) => a.Intersect(b))
    .SingleOrDefault();

var testInput = """
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
""";
var testInputLines = testInput.Split('\n');
Console.WriteLine(GetColOfReflection(testInputLines));

5


In [8]:
var testInput2 = """
#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
""";
var testInputLines2 = testInput2.Split('\n');

// Should be no common mirror line in the row direction (answer = 0);
var colAnswer2 = GetColOfReflection(testInputLines2);
Console.WriteLine(colAnswer2);

var rowsEqual = (string left, string right) => left == right;
int GetRowOfReflection(IList<string> inputLines) => GetMirrors(inputLines, rowsEqual).SingleOrDefault();

// This pattern reflects across the horizontal line between rows 4 and 5.
var rowAnswer2 = GetRowOfReflection(testInputLines2);
Console.WriteLine(rowAnswer2);


0
4


In [9]:
IEnumerable<List<string>> ParsePuzzles(string[] inputLines) {
    var result = new List<string>();

    foreach (var line in inputLines) {
        if (line.Length == 0) {
            yield return result;
            result = new();
        } else {
            result.Add(line);
        }
    }

    if (result.Any()) {
        yield return result;
    }
}
var inputPuzzles = ParsePuzzles(inputLines).ToArray();
Console.WriteLine(inputPuzzles.Length);
Console.WriteLine(string.Join('\n', inputPuzzles[0]));

100
######..#
.##....#.
.....##.#
.....##.#
.##..#.#.
######..#
#..#..###
.##..#.##
#####.#.#
#..#.##..
.....###.
#####....
....##.#.


In [10]:
// To summarize your pattern notes, add up the number of columns to the left of
// each vertical line of reflection; to that, also add 100 multiplied by the number
// of rows above each horizontal line of reflection. In the above example, the
// first pattern's vertical line has 5 columns to its left and the second pattern's
// horizontal line has 4 rows above it, a total of 405.

var leftColumns = inputPuzzles.Select(GetColOfReflection).Sum();
Console.WriteLine(leftColumns);
var aboveRows = inputPuzzles.Select(GetRowOfReflection).Sum();
Console.WriteLine(aboveRows);

var part1Answer = leftColumns + aboveRows * 100;
Console.WriteLine(part1Answer);

335
317
32035


In [11]:
// 32035 is correct!
Ensure(32035, part1Answer);

In [12]:
// --- Part Two ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/13

In [13]:
char Swap(char ch) => ch switch {
    '#' => '.',
    '.' => '#',
    _ => throw new Exception("Unexpected char")
};

In [14]:
IEnumerable<List<string>> SwapAllChars(List<string> puzzleInput) {
    for (var row = 0; row < puzzleInput.Count; row++) {
        var line = puzzleInput[row].ToCharArray();
        for (var col = 0; col < puzzleInput[0].Length; col++) {
            line[col] = Swap(line[col]);
            puzzleInput[row] = new string(line);
            yield return puzzleInput;
            line[col] = Swap(line[col]);
        }
        puzzleInput[row] = new string(line);
    }
}

// 9 x 13 = 117
Console.WriteLine(SwapAllChars(inputPuzzles[0]).Count());

117


In [15]:
ImmutableHashSet<int> GetRowOfReflection2(IList<string> inputLines) => GetMirrors(inputLines, rowsEqual);

ImmutableHashSet<int> GetColOfReflection2(IList<string> inputs) => inputs
    .Select(input => GetMirrors(input.ToList(), charsEqual))
    .Aggregate((a, b) => a.Intersect(b));

(int newColLeft, int newRowAbove) GetNewSplit(List<string> inputPuzzle) {
    int originalColLeft = GetColOfReflection(inputPuzzle);
    int originalRowAbove = GetRowOfReflection(inputPuzzle);

    var newColLeft = SwapAllChars(inputPuzzle).SelectMany(GetColOfReflection2).ToHashSet();
    newColLeft.Remove(originalColLeft);
    var newRowAbove = SwapAllChars(inputPuzzle).SelectMany(GetRowOfReflection2).ToHashSet();
    newRowAbove.Remove(originalRowAbove);
    
    return (newColLeft.SingleOrDefault(), newRowAbove.SingleOrDefault());
}

In [16]:
var testInput2 = """
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
""";
var testInputLines2 = testInput2.Split('\n');
var testInputPuzzles2 = ParsePuzzles(testInputLines2).ToList();
Console.WriteLine("Original puzzle:");
Console.WriteLine(string.Join('\n', testInputPuzzles2[0]));

(int colsLeft, int rowsAbove) GetRowCols(IList<string> puzzleInput) => (GetColOfReflection(puzzleInput), GetRowOfReflection(puzzleInput));
Console.WriteLine(GetRowCols(testInputPuzzles2[0]));

Console.WriteLine("New split: ");
Console.WriteLine(GetNewSplit(testInputPuzzles2[0]));

// The first pattern's smudge is in the top-left corner. If the top-left # were instead ., it would have a different, horizontal line of reflection:

// 1 ..##..##. 1
// 2 ..#.##.#. 2
// 3v##......#v3
// 4^##......#^4
// 5 ..#.##.#. 5
// 6 ..##..##. 6
// 7 #.#.##.#. 7


Original puzzle:
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
(5, 0)
New split: 
(0, 3)


In [17]:
// In each pattern, fix the smudge and find the different line of reflection.
// What number do you get after summarizing the new reflection line in each pattern
// in your notes?

var allNewSplits = inputPuzzles.Select(GetNewSplit).ToList();
var answer2ColsLeft = allNewSplits.Select(s => s.newColLeft).Sum();
var answer2RowsAbove = allNewSplits.Select(s => s.newRowAbove).Sum();

var part2Answer = answer2ColsLeft + answer2RowsAbove * 100;
Console.WriteLine(part2Answer);

24847


In [18]:
// 24847 is correct!
Ensure(24847, part2Answer);